## Converting your .pt file to a tensorflow.js file

In [ ]:
!pip install tensorflowjs tensorflow ultralytics

In [ ]:
import os
from ultralytics import YOLO
import shutil
import tensorflow as tf
from google.colab import files as colab_files

def find_saved_model(base_path):
    """Find the SavedModel directory in the export path"""
    for root, dirs, filenames in os.walk(base_path):
        if 'saved_model.pb' in filenames:
            return root
    return None

# def add_signatures(saved_model_dir):
#     """Load the SavedModel and add required signatures"""
#     print("Adding signatures to SavedModel...")

#     # Load the model
#     model = tf.saved_model.load(saved_model_dir)

#     # Create a wrapper function that matches the model's interface
#     @tf.function(input_signature=[
#         tf.TensorSpec(shape=[1, 640, 640, 3], dtype=tf.float32, name='images')
#     ])
#     def serving_fn(images):
#         # Pass False for training parameter
#         return model(images, False, None)

#     # Convert the model
#     concrete_func = serving_fn.get_concrete_function()

#     # Create a new SavedModel with the signature
#     tf.saved_model.save(
#         model,
#         saved_model_dir,
#         signatures={
#             'serving_default': concrete_func
#         }
#     )

#     print("Signatures added successfully")
#     return saved_model_dir


def add_signatures(saved_model_dir):
    """Load the SavedModel and add required signatures"""
    print("Adding signatures to SavedModel...")

    # Load the model
    model = tf.saved_model.load(saved_model_dir)

    # Create a wrapper function that properly handles the model's internal expectations
    @tf.function(input_signature=[
        tf.TensorSpec(shape=[1, 640, 640, 3], dtype=tf.float32, name='images')
    ])
    def serving_fn(images):
        # YOLO models often need specific preprocessing
        # Normalize inputs to 0-1 if needed (though your JS code also does this)
        normalized_images = images

        # Call the model with correct parameters
        # The issue might be that your model expects different parameters
        # or that the conversion process changed the expected signature
        result = model(normalized_images, training=False)

        # Make sure outputs match what your JS code expects
        if isinstance(result, dict):
            # Some models return dictionaries
            return result
        else:
            # Format output consistently for your JS expecting an array
            return {"output": result}

    # Create a new SavedModel with the signature
    tf.saved_model.save(
        model,
        saved_model_dir,
        signatures={
            'serving_default': serving_fn
        }
    )

    print("Signatures added successfully")
    return saved_model_dir

def convert_to_tfjs(pt_model_path, output_dir):
    """
    Convert a PyTorch YOLO model to TensorFlow.js format

    Args:
        pt_model_path (str): Path to the .pt file
        output_dir (str): Directory to save the converted model
    """
    try:
        # Ensure output directory exists
        os.makedirs(output_dir, exist_ok=True)

        # Load the model
        print(f"Loading YOLO model from {pt_model_path}...")
        model = YOLO(pt_model_path)

        # First export to TensorFlow format
        print("Exporting to TensorFlow format...")


        success = model.export(
            format='saved_model',
            imgsz=640,
            half=False,
            simplify=False
        )

        # Find the SavedModel directory
        saved_model_dir = find_saved_model(os.path.join(os.getcwd(), "best_saved_model"))
        if not saved_model_dir:
            raise Exception(f"Cannot find SavedModel directory in {os.path.dirname(pt_model_path)}")

        print(f"Found SavedModel at: {saved_model_dir}")

        # Add signatures to the model
        saved_model_dir = add_signatures(saved_model_dir)

        # Convert to TensorFlow.js
        print("Converting to TensorFlow.js format...")
        tfjs_target_dir = os.path.join(output_dir, 'tfjs_model')

        # Ensure clean target directory
        if os.path.exists(tfjs_target_dir):
            shutil.rmtree(tfjs_target_dir)
        os.makedirs(tfjs_target_dir)

        # Try conversion with modified parameters
        conversion_command = (
            f"tensorflowjs_converter "
            f"--input_format=tf_saved_model "
            f"--output_format=tfjs_graph_model "
            f"--saved_model_tags=serve "
            f"--control_flow_v2=True "
            f"'{saved_model_dir}' "
            f"'{tfjs_target_dir}'"
        )

        print(f"Running conversion command: {conversion_command}")
        result = os.system(conversion_command)

        if result != 0:
            raise Exception("TensorFlow.js conversion failed")

        # Verify conversion
        if not os.path.exists(os.path.join(tfjs_target_dir, 'model.json')):
            raise Exception("TensorFlow.js conversion failed - model.json not found")

        print(f"Successfully converted model to TensorFlow.js format")
        print(f"Output saved to: {tfjs_target_dir}")

        # Print model files
        print("\nConverted model files:")
        for filename in os.listdir(tfjs_target_dir):  # Renamed 'file' to 'filename'
            print(f"- {filename}")

        # Create a zip file of the converted model
        zip_path = f"{tfjs_target_dir}.zip"
        shutil.make_archive(tfjs_target_dir, 'zip', tfjs_target_dir)

        # Download the zip file using the renamed colab_files module
        colab_files.download(zip_path)

    except Exception as e:
        print(f"Error during conversion: {str(e)}")
        print("\nDebug information:")
        print(f"Current working directory: {os.getcwd()}")
        print(f"PT model exists: {os.path.exists(pt_model_path)}")
        if 'saved_model_dir' in locals():
            print(f"SavedModel directory exists: {os.path.exists(saved_model_dir)}")
            if os.path.exists(saved_model_dir):
                print("SavedModel contents:")
                for root, dirs, filenames in os.walk(saved_model_dir):  # Renamed 'files' to 'filenames'
                    print(f"\nDirectory: {root}")
                    for filename in filenames:  # Renamed 'f' to 'filename'
                        print(f"  - {filename}")
        raise

# Usage
from google.colab import files as colab_files  # Use consistent naming
uploaded = colab_files.upload()
pt_model_path = next(iter(uploaded.keys()))
output_dir = "converted_model"
convert_to_tfjs(pt_model_path, output_dir)